In [30]:
import pandas as pd
import numpy as np 
import re


pthurl = '3rdpartyExport.csv'

df = pd.read_csv(pthurl)

tgt_fe_expects = ['countertops', 'AC_furnace_age', 'roof_age', 'flooring', 'windows', 'solar', 'pole_barn', 'swimming_pool']

print(df.shape)

df.columns

#[x for x in df.columns if 'sol' in x]

#df[['SqFt-Est Tot Fin','SqFt-Est Fin Lower Floor', 'SqFt-Est Fin Abv Grd', 'SqFt-Est Tot Bsmt']]

# WE WANT: 
we_want = ['SqFt-Est Tot Fin', 'Beds Total', 'Baths.Lavs']


# df['Architecture Style'].value_counts()

df.dtypes

(955, 44)


 $/ sq ft                         object
Sale Price                         int64
List Price                         int64
>90 Day Before Sale Zestimate     object
$ / sq ft for keywords            object
Close Date                        object
Street Number                      int64
Street Name                       object
Street Suffix                     object
City                              object
County                            object
Zip5                               int64
Unnamed: 12                      float64
Beds Total                         int64
Baths.Lavs                       float64
Est Fin Abv Grd SqFt               int64
Acreage                          float64
Architecture Level                object
Architecture Style                object
 $/ sq ft .1                      object
Basement Type                     object
Unnamed: 21                      float64
Sale Price.1                       int64
Exterior                          object
Cooling         

In [28]:
def clean(dat: pd.DataFrame) -> pd.DataFrame: 
    ''''''
    def try_lower(x): 
        ''''''
        try: 
            return x.lower()
        except AttributeError:
            return x

    def try_float_cash(x): 
        ''''''
        try:
            return float('.'.join(re.findall(r'\d+', x)))
        except TypeError: 
            return x
        except ValueError: 
            return x

        
    return (dat.assign(**{feat: dat[feat].apply(try_lower).apply(try_float_cash)
                          for feat in dat.select_dtypes(exclude=[np.number]).columns})
            .drop([x for x in dat.columns if dat[x].isna().sum()>0], axis=1)
            .rename(columns={' $/ sq ft ': 'price_per_sq_ft', 
                               '$ / sq ft for keywords': 'price_per_sq_ft_by_keyword', 
                               ' $/ sq ft .1': 'price_per_sq_ft_.1', 
                            'Baths.Lavs': 'baths_total', 
                             'SqFt-Est Tot Fin': 'home_size'
                            })
            .rename(columns=lambda s: s.replace(' ', '_').lower().replace(':', ''))
            
           )

# target: sale_price
df = clean(pd.read_csv(pthurl))

all([x==y for x,y in df[['price_per_sq_ft', 'price_per_sq_ft_.1']].values])
assert df.isna().sum().sum()==0
all([x==y for x,y in df[['sale_price', 'sale_price.1']].values])

print(df.shape)

#df.columns
df.head()

(955, 23)


,price_per_sq_ft,sale_price,list_price,close_date,street_number,street_name,city,county,zip5,beds_total,...,price_per_sq_ft_.1,sale_price.1,original_list_price,photo_count,pool_yn,avg,count,sqft-est_tot_fin,sqft-est_fin_abv_grd,year_built
0,98.0,170000,178000,04/16/2018,1013,genesee,royal oak,oakland,48073,3,...,98.0,170000,199900,11,False,200.0,154,1.726,1.026,1952
1,99.0,181000,179900,08/22/2018,3266,harvard,royal oak,oakland,48073,3,...,99.0,181000,199900,2,False,197.0,154,1.910,1.210,1949
2,103.0,165500,199900,12/03/2018,708,campbell,royal oak,oakland,48067,4,...,103.0,165500,199900,21,False,217.0,154,1.764,1.764,1923
3,105.0,290000,300000,06/04/2018,2607,carman,royal oak,oakland,48073,4,...,105.0,290000,325000,31,False,232.0,154,936.000,936.000,1953
4,105.0,125000,159900,04/23/2018,2931,rochester,royal oak,oakland,48073,3,...,105.0,125000,174999,25,False,155.0,3,1.792,1.064,1949


In [29]:
'''User_Provided_Data: {

countertops: 
AC_furnace_age:
roof_age:
flooring:
windows:
solar:
pole_barn:
swimming_pool:    '''


df.dtypes

# pick 8, numerics

# target: sale_price

# predictors: ['price_per_sq_ft', 'beds_total', 'baths_total (baths.lavs)', 
#           'original_list_price', year_built', 'sqft-est_tot_fin', 'sqft-est_fin_abv_grd', 'acreage']

price_per_sq_ft         float64
sale_price                int64
list_price                int64
close_date               object
street_number             int64
street_name              object
city                     object
county                   object
zip5                      int64
beds_total                int64
baths.lavs              float64
est_fin_abv_grd_sqft      int64
acreage                 float64
price_per_sq_ft_.1      float64
sale_price.1              int64
original_list_price       int64
photo_count               int64
pool_yn                    bool
avg                     float64
count                     int64
sqft-est_tot_fin        float64
sqft-est_fin_abv_grd    float64
year_built                int64
dtype: object

In [16]:
df[['sqft-est_tot_fin', 'sqft-est_fin_abv_grd']]
'''
sqft-est_tot_fin : square foot estimate total finished 

'''
df[['beds_total', 'baths.lavs', 'sqft-est_tot_fin']]

,beds_total,baths.lavs,sqft-est_tot_fin
0,3,2.0,1.726
1,3,1.1,1.910
2,4,1.0,1.764
3,4,3.0,936.000
4,3,1.0,1.792
5,3,1.2,949.000
6,4,2.1,1.835
7,3,1.1,1.193
8,2,1.1,1.811
9,3,1.0,1.010


In [17]:
print(df.columns)

Index(['price_per_sq_ft', 'sale_price', 'list_price', 'close_date',
       'street_number', 'street_name', 'city', 'county', 'zip5', 'beds_total',
       'baths.lavs', 'est_fin_abv_grd_sqft', 'acreage', 'price_per_sq_ft_.1',
       'sale_price.1', 'original_list_price', 'photo_count', 'pool_yn', 'avg',
       'count', 'sqft-est_tot_fin', 'sqft-est_fin_abv_grd', 'year_built'],
      dtype='object')


In [66]:
df['baths.lavs']

0      2.0
1      1.1
2      1.0
3      3.0
4      1.0
5      1.2
6      2.1
7      1.1
8      1.1
9      1.0
10     1.0
11     1.1
12     1.1
13     2.0
14     2.0
15     2.0
16     2.0
17     2.0
18     2.0
19     2.1
20     2.0
21     1.0
22     2.0
23     2.1
24     1.1
25     1.0
26     1.0
27     1.1
28     2.1
29     2.0
      ... 
925    1.1
926    2.0
927    2.1
928    2.0
929    2.0
930    2.1
931    1.1
932    1.1
933    2.0
934    2.0
935    3.0
936    2.0
937    1.0
938    2.0
939    2.0
940    1.1
941    2.0
942    2.1
943    2.1
944    2.0
945    2.1
946    2.0
947    2.0
948    2.0
949    2.0
950    1.1
951    2.0
952    4.1
953    2.1
954    2.0
Name: baths.lavs, Length: 955, dtype: float64

In [19]:
# Peyton says Doc2Vec from GenSim is what we want for this
# df.public_remarks

In [12]:
import re
pat = r"^[$]{1}[\s]{,2}[0-9.]+"

re.match(pat, '$ 45.3')

<_sre.SRE_Match object; span=(0, 6), match='$ 45.3'>

In [25]:
list(filter(str.isdigit, '$ 45.3'))

['4', '5', '3']